# Importing libraries

In [ ]:
libraries_dict = {
    'utility': [
        'import os',
        'import sys',
        'import numpy as np',
        'import pandas as pd',
    ],
        
    'deep_learning':[
        'import tensorflow as tf',
        'import keras as k',
    ],   
        
    'statistics':[
        'import scipy as sp',
    ],    
        
    'display':[
        'from tqdm import tqdm_notebook',
        'from IPython.core.display import display, HTML',
    ],     
    
    'image_processing': [
        'from PIL import Image',
        'import cv2 as cv', 
        'import scipy.ndimage as ndi',
        'from skimage import measure',
        'from skimage import transform',
    ],
    
    'medical_imaging': [
        'import pydicom', 
        'import nibabel as nib', 
        'import dicom2nifti', 
    ]
}

In [ ]:
def import_libraries(topics = ['utility']):
    
    if topics == 'all':
        topics = list(libraries_dict.keys())
    
    libraries = [';'.join(libraries_dict[topic]) for topic in topics]
    
    return ';'.join(libraries)

# Pre-processing functions

In [ ]:
def get_bounding_box_coordinate_dict(bounding_boxes, accession_number, sequence_name, rois = ['tumor']):
    coordinate_dict = {}
    
    for roi in rois:

        bounding_boxes_acc = bounding_boxes[(bounding_boxes['accession_number'] == accession_number) & (bounding_boxes['roi'] == roi)]
        roi_indices = bounding_boxes_acc['roi_idx']

        bounding_boxes_acc = bounding_boxes_acc[[col for col in bounding_boxes_acc.columns if sequence_name in col]]

        if consider_biggest_slice_coordinates == False:
            bounding_boxes_acc = bounding_boxes_acc.drop(columns = [col for col in bounding_boxes_acc.columns if 'big' in col])

        bounding_boxes_acc.columns = bounding_boxes_acc.columns = [col.split('_')[-1][0] for col in bounding_boxes_acc.columns]

        for roi_idx in range(len(roi_indices)):
            coordinate_values_roi = pd.DataFrame(bounding_boxes_acc.iloc[roi_idx,:]).T
            coordinate_values_roi = [coordinate_values_roi.loc[:,bounding_boxes_acc.columns == plane].values[0] for plane in bounding_box_planes]
            coordinate_dict_roi = dict(zip(bounding_box_planes, coordinate_values_roi))

            coordinate_dict[str(roi) + str(roi_idx)] = coordinate_dict_roi
            
    return coordinate_dict

def extract_ROI(img, roi_coordinate_dict, bounding_boxes_delta_dictionary = None):
    
    for plane, coordinates in roi_coordinate_dict.items():
        
        try:
            delta = bounding_boxes_delta_dictionary[plane].copy()   

            delta[0] = roi_coordinate_dict[plane][0] if (roi_coordinate_dict[plane][0] - delta[0] < 0) else delta[0] 
            if plane == 'z': delta[1] = img.shape[0] - roi_coordinate_dict[plane][1] -1 if roi_coordinate_dict[plane][1] + delta[1] > img.shape[0] else delta[1] 
            if plane == 'y': delta[1] = img.shape[1] - roi_coordinate_dict[plane][1] -1 if roi_coordinate_dict[plane][1] + delta[1] > img.shape[1] else delta[1] 
            if plane == 'x': delta[1] = img.shape[2] - roi_coordinate_dict[plane][1] -1 if roi_coordinate_dict[plane][1] + delta[1] > img.shape[2] else delta[1] 

        except:
            delta = 0
        
        if plane == 'z': img = img[range(roi_coordinate_dict['z'][0] - delta[0], roi_coordinate_dict['z'][1]+1 + delta[1]), :, :]
        if plane == 'y': img = img[:, range(roi_coordinate_dict['y'][0] - delta[0], roi_coordinate_dict['y'][1]+1 + delta[1]), :]
        if plane == 'x': img = img[:, :, range(roi_coordinate_dict['x'][0] - delta[0], roi_coordinate_dict['x'][1]+1 + delta[1])]

    return img

def pre_process_image_to_vgg(image):

    image = np.swapaxes(image,0,2)
    image = image.astype(np.float32)
    image = cv.resize(image,(224,224))
    image = preprocess_input(image)

    return image

def floor_dimenssions(image_list):
    dimensions = pd.DataFrame([img.shape for img in image_list])
    unique_dimensions, counts = np.unique(dimensions, axis = 0, return_counts = True)
    min_dimensions = tuple(dimensions.min(axis = 0).values)
    
    return np.array(unique_dimensions), np.array(min_dimensions)


def pool_intensities(img, n_slices_raw_image, slice_thickness_raw_image, slices_per_batch  = 10, overlap = 0, operator = 'iqr', method = 'relative', verbose = 0):
    if method == 'relative':
        depth_mm = slice_thickness_raw_image*n_slices_raw_image
        slices_per_batch = depth_mm*(slices_per_batch)
        slices_per_batch = int(np.round(a = slices_per_batch/slice_thickness_raw_image, decimals = 0))
        overlap = depth_mm*(overlap)
        overlap = int(np.round(a = overlap/slice_thickness_raw_image, decimals = 0))
    
    n_original_slices = img.shape[0]

    start = 0
    stop = slices_per_batch
    batched_images = []
    while True:
        batched_images.append(img[start:stop])
        start = start + slices_per_batch-overlap
        stop = start + slices_per_batch
        if (stop > n_original_slices) & (start < n_original_slices):
            batched_images.append(img[start:n_original_slices])
            break
        
        if (start == n_original_slices):
            break
            
    if operator == 'iqr':
        mips = np.array([np.percentile(b, q=0.75, axis = 0) - np.percentile(b, q=0.25, axis = 0) for b in batched_images])
    
    if verbose >0:
        visualize_mri_slices(img_array = mips, ncols = 5, bounding_box_coordinates_dict = None)
    return mips

def get_pooled_image_sequences(key_values, images, filter_size, filter_overlap, pool_op):
    images_pooled = []
    for key in tqdm_notebook(key_values):    
        seuquence_images = images[key].copy()
        n_slices_raw_image = metadata_dicom_headers[metadata_dicom_headers[key_subject] == key]['depth_pixel'].values[0]
        slice_thickness_raw_image = metadata_dicom_headers[metadata_dicom_headers[key_subject] == key]['slice_thickness'].values[0]
        
        seuquence_images_pooled = np.array([pool_intensities(img = img, slices_per_batch  = filter_size, overlap = filter_overlap, operator = pool_op, method = 'relative', n_slices_raw_image = n_slices_raw_image, slice_thickness_raw_image=slice_thickness_raw_image) for img in seuquence_images])

        batched_sequences = np.array([seuquence_images_pooled[:,idx,:,:] for idx in range(seuquence_images_pooled.shape[1])])

        images_pooled.append(np.stack(batched_sequences, axis = 0))
        
    return images_pooled


def get_vgg_embeddings(images):
    embeddings = []
    for stacked_batches in tqdm_notebook(images):  
        instance_embeddings = get_image_embeddings.predict(np.stack([pre_process_image_to_vgg(s) for s in list(stacked_batches)], axis = 0))
        embeddings.append(instance_embeddings)

    return embeddings


def get_patient_label(patient_date_recurrence, patient_date_followup, patient_date_treatment, T):
    patient_time_to_recurrence = (patient_date_recurrence -patient_date_treatment).dt.days.values[0]
    patient_followup_time = (patient_date_followup -patient_date_treatment).dt.days.values[0]

    patient_recurrence = False if  np.isnan(patient_time_to_recurrence) else True
    
    if (patient_recurrence == True) & (patient_time_to_recurrence <= T): return True 
    elif patient_followup_time >= T: return False 
    return None 

def get_cohort_by_time_stamp(metadata_labels, T, verbose = 1):
    cohort_data = {}
    
    indices, accession_numbers, labels = [], [], []
    for idx, accession_number in enumerate(metadata_labels[key_subject]):
        patient_values = metadata_labels[metadata_labels[key_subject] == accession_number]
        patient_label =  get_patient_label(patient_date_recurrence = patient_values['date_recurrence'], 
                                           patient_date_followup =  patient_values['date_followup'],
                                           patient_date_treatment =  patient_values['date_treatment'], 
                                           T = T*365)
        if patient_label != None:
            
            indices.append(idx)
            accession_numbers.append(accession_number)
            labels.append(patient_label)
        
    if verbose > 0:
        print('Time to recurrence: {}'.format(T))
        print('Label Distribution: {}'.format(np.unique(labels, return_counts=True)))
        
    return np.array(indices), np.array(accession_numbers), np.array(labels)


def get_cohort_features(key_subject, key_values, metadata_feature_values, metadata_clinical_metadata, modalities):
    feature_names = [name for (name, modality) in metadata_clinical_metadata[['name', 'modality']].values if modality in modalities]
    features_cohort = metadata_feature_values_clinical[metadata_feature_values[key_subject].isin(key_values)].copy()

    return np.array(features_cohort.index.tolist()), np.array(features_cohort[key_subject].values.tolist()), features_cohort[feature_names].values, np.array(feature_names)


def aggregate_embeddings(embedding_dict, keys):

    aggregated_embeddings = []
    for key_subject_outer in keys:
        feature_values_imaging_subject = embedding_dict[key_subject_outer].copy()
        feature_values_imaging_subject = np.max(feature_values_imaging_subject,  axis = 0)
        aggregated_embeddings.append(feature_values_imaging_subject)
        
    return np.array(aggregated_embeddings)

In [ ]:
import shutil
import deepdish as dd
from itertools import product, combinations
from xgboost import XGBClassifier
exec(import_libraries(topics = ['utility', 'display', 'medical_imaging', 'image_processing']))
from scipy import ndimage
from sklearn.model_selection import LeaveOneOut, StratifiedShuffleSplit
from sklearn.metrics import roc_auc_score
from keras.models import Model
from keras.applications.vgg16 import VGG16, preprocess_input
from sklearn.preprocessing import LabelEncoder
display(HTML("<style>.container { width:100% !important; }</style>"))

# Data paths

In [ ]:
path_root = r'XXXX'
path_images = os.path.join(path_root, 'data/images')
path_metadata = os.path.join(path_root, 'data/metadata')
path_metadata_clinical = os.path.join(path_metadata, 'clinical')
path_metadata_bounding_boxes = os.path.join(path_metadata, 'bounding_boxes')
path_metadata_dicom_headers = os.path.join(path_metadata, 'dicom_images')

In [ ]:
key_subject = 'accession_number'
sequence_names_ordered = ['arterial', 'portal' ,'delayed']
roi = 'liver0'
model = VGG16()
get_image_embeddings = Model(inputs=model.input, outputs=model.get_layer('fc2').output)
encoding_method = 'integer'

# Metadata

In [ ]:
metadata_dicom_headers = pd.read_csv(filepath_or_buffer = os.path.join(path_metadata_dicom_headers, 'metadata.csv'), index_col = 0)
metadata_bounding_boxes = pd.read_csv(filepath_or_buffer = os.path.join(path_metadata_bounding_boxes, 'bounding_boxes_processed.csv'))
metadata_clinical_metadata =  pd.read_csv(filepath_or_buffer = os.path.join(path_metadata_clinical, 'clinical_metadata.csv'), index_col = 0)
metadata_clinical_values =  pd.read_csv(filepath_or_buffer = os.path.join(path_metadata_clinical, 'clinical_values.csv'), index_col = 0)

metadata_feature_values_clinical = metadata_clinical_values[[key_subject] + list(metadata_clinical_metadata[metadata_clinical_metadata['modality'] != 'target']['name'].values)].copy()
metadata_label_values = metadata_clinical_values[[key_subject] + list(metadata_clinical_metadata[metadata_clinical_metadata['modality'] == 'target']['name'].values)].copy()

for name in ['date_treatment', 'date_recurrence', 'date_followup']: 
    metadata_label_values[name] = pd.to_datetime(metadata_label_values[name])
metadata_label_values['time_to_recurrence'] = (metadata_label_values['date_recurrence'] - metadata_label_values['date_treatment']).dt.days
metadata_label_values['followup_time'] = (metadata_label_values['date_followup'] - metadata_label_values['date_treatment']).dt.days

metadata_features_categorical = metadata_clinical_metadata[(metadata_clinical_metadata['data_type'] == 'categorical') | 
                                                           (metadata_clinical_metadata['data_type'] == 'ordinal') ].copy()

for name, modality in metadata_features_categorical[['name', 'modality']].values:
    metadata_feature_values_clinical[name] = [val.replace(' ', '')  if type(val) == str else val for val in metadata_feature_values_clinical[name]]
    feature_values = metadata_feature_values_clinical[name].copy()
        
    if encoding_method == 'integer':
        le = LabelEncoder()
        feature_values_nans = feature_values.isna()
        feature_values_encoded = le.fit_transform(feature_values.astype(str))
        feature_values_encoded = [None if nan else val for (val, nan) in zip(feature_values_encoded,feature_values_nans) ]
        metadata_feature_values_clinical[name] = feature_values_encoded

metadata_key_values = metadata_label_values[key_subject].values.copy()

# Images

In [ ]:
new_shape_z = None
images = {}

for key in tqdm_notebook(metadata_key_values):    
    seuquence_images = [dd.io.load(os.path.join(path_images, str(key), sequence  ,'numpy_image_' + roi + '.h5')) for sequence in sequence_names_ordered]
    
    old_shapes, new_shape = floor_dimenssions(image_list = seuquence_images)
    if new_shape_z != None: new_shape[0] = new_shape_z
            
    seuquence_images_processed = [ndimage.zoom(img, np.array(new_shape)/np.array(img.shape)) for img in seuquence_images]
    images[key] = seuquence_images_processed

# Pooling and extraction of image embeddings

In [ ]:
filter_size = np.arange(start=0.05, stop=0.40, step=0.05)
filter_overlap = np.arange(start=0.03, stop=0.18, step=0.03)
filters = np.array(list(product(filter_size, filter_overlap)))
np.random.seed(1690811) 
filter_random_idx = np.random.choice(a=range(len(filters)), size=4, replace = False)
pool_filter = filters[filter_random_idx]
pool_filter = [tuple(f.astype(np.float16)) for f in pool_filter]

In [ ]:
pool_operator = ['iqr']

embedding_dict = {}
for c in pool_filter:
    for p in pool_operator:
        print(c,p)
        combination_pooled_images = get_pooled_image_sequences(key_values = metadata_key_values,
                                                               images = images,
                                                               filter_size = c[0], 
                                                               filter_overlap = c[1], 
                                                               pool_op = p)
        
        combination_embeddings = get_vgg_embeddings(images = combination_pooled_images)
        
        embedding_dict[(c, p)] = dict(zip(metadata_key_values, combination_embeddings))

# Training Pipeline

### Hyper Parameters 

In [ ]:
splitting_points = [1,2,3,4,5,6] 

feature_sets =  [['clinical'], ['clinical', 'vgg'], ['vgg']]

hp_dict = {
     'n_estimators': np.arange(start=80, stop=85, step=10),
     'max_depth': np.arange(start=5, stop=7, step = 5).astype(int),
     'gamma':  np.arange(start=0, stop=0.05, step=0.1),
     'min_child_weight': np.arange(start=1.5, stop=2, step=1),
     'max_delta_step': np.arange(start=2, stop=2.5, step=1),
     'subsample':np.arange(start=1, stop= 1.1, step= 0.2),
     'colsample_bytree': np.arange(start=0.8, stop=1.1, step=1),
     'pool_filter': np.array(pool_filter),
     'pool_operator': np.array(pool_operator),
     'embedding_aggregation_operator': np.array(['max'])
      }

np.random.seed(123)
for key, value in hp_dict.items():
    if key == 'pool_filter': continue
    sampled_hp_idx = np.random.choice(a = range(len(value)), size = 1, replace = False)
    sampled_hp_values = np.array(value)[sampled_hp_idx]
    hp_dict[key] = sampled_hp_values

hp_keys = list(hp_dict.keys())
hp_instruction = 'product(' + ', '.join(['hp_dict[hp_keys[' + str(i) + ']]' for i in range(len(hp_keys))]) + ')'

In [ ]:
final_results_preliminary_list = []
    
for modalities in tqdm_notebook(feature_sets, 'Modality'): 
    for t in tqdm_notebook(splitting_points, 'T'):
        labels_indices, labels_key_values, labels_values = get_cohort_by_time_stamp(metadata_labels = metadata_label_values, T=t, verbose = 0)        

        features_indices, features_key_values, feature_values, feature_names = get_cohort_features(key_subject = key_subject, 
                                                                                                   key_values = labels_key_values, 
                                                                                                   metadata_feature_values = metadata_feature_values_clinical, 
                                                                                                   metadata_clinical_metadata = metadata_clinical_metadata, 
                                                                                                   modalities = modalities)        

        if np.all(labels_key_values == features_key_values):
            key_values = labels_key_values.copy()
            N=len(key_values)

            print('Time Stamp:            {}'.format(t))
            
        else:
            print('Error! features order doesnt match labels order')
        number_negative_instances = np.unique(labels_values, return_counts = True)[1][0]
        number_positive_instances = np.unique(labels_values, return_counts = True)[1][1]
        scale_pos_weight = number_negative_instances / number_positive_instances

        np.random.seed(123)
        indices_shuffled = np.random.choice(a = range(N), size = N, replace = False)
        labels_shuffled = labels_values[indices_shuffled].copy()
        key_values_shuffled = key_values[indices_shuffled].copy()
        feature_values_shuffled = feature_values[indices_shuffled, :].copy()
        
        loo = LeaveOneOut()
        cv_splits_outer = list(loo.split(labels_shuffled, labels_shuffled)) 

        print('Fold Outer: ', end = '')
        for fold_outer, (train_idx_outer, test_idx_outer) in tqdm_notebook(enumerate(cv_splits_outer), 'Outer'):

            train_features_outer = feature_values_shuffled[train_idx_outer,:].copy()
            train_labels_outer = labels_shuffled[train_idx_outer].copy()
            train_key_values_outer = key_values_shuffled[train_idx_outer].copy()

            test_features_outer = feature_values_shuffled[test_idx_outer,:].copy()
            test_labels_outer = labels_shuffled[test_idx_outer].copy()
            test_key_values_outer = key_values_shuffled[test_idx_outer].copy()   

            sss = StratifiedShuffleSplit(n_splits=6, test_size=int(len(train_idx_outer)*0.1), random_state=123)
            cv_splits_inner = list(sss.split(X=train_labels_outer,y= train_labels_outer))      

            best_hyperparameters_combinaion_dict = {
                'hyperparameters': None,
                'metric_inner_loop': 0
             }
            
            for hyperparameters in eval(hp_instruction):
                train_features_combined_values_outer = train_features_outer.copy()
                
                hyperparameters = dict(zip(hp_keys, hyperparameters)) 
                hyperparameters_metric_inner_loop = 0                            
                
                if 'vgg' in modalities:
                    feature_values_imaging_dict = embedding_dict[(tuple(hyperparameters['pool_filter']), hyperparameters['pool_operator'])]
                    train_features_imaging_values_outer = aggregate_embeddings(embedding_dict = feature_values_imaging_dict, keys = train_key_values_outer)
                    train_features_combined_values_outer = np.concatenate([train_features_combined_values_outer, train_features_imaging_values_outer], axis = 1)
    
                for fold_inner, (train_idx_inner, test_idx_inner) in enumerate(cv_splits_inner):

                    train_features_inner = train_features_combined_values_outer[train_idx_inner,:].copy()
                    train_labels_inner = train_labels_outer[train_idx_inner].copy()
                    train_key_values_inner = train_key_values_outer[train_idx_inner].copy()

                    test_features_inner = train_features_combined_values_outer[test_idx_inner,:].copy()
                    test_labels_inner = train_labels_outer[test_idx_inner].copy()
                    test_key_values_inner = train_key_values_outer[test_idx_inner].copy()
                    
 
                    predictor = XGBClassifier(
                        n_estimators =  hyperparameters['n_estimators'],
                        max_depth = hyperparameters['max_depth'],
                        gamma = hyperparameters['gamma'],
                        min_child_weight = hyperparameters['min_child_weight'],
                        max_delta_step = hyperparameters['max_delta_step'],
                        subsample = hyperparameters['subsample'],
                        colsample_bytree = hyperparameters['colsample_bytree'],
                        scale_pos_weight = scale_pos_weight,
                        n_jobs= 8,
                        random_state = 373315
                        ) 

                    predictor.fit(X = train_features_inner, y = train_labels_inner)
                    y_pred_inner = predictor.predict(test_features_inner) 
                    y_pred_proba_inner = predictor.predict_proba(test_features_inner) 
                    metric_fold = roc_auc_score(y_true=test_labels_inner, y_score=[p[1] for p in y_pred_proba_inner])
                    
                    hyperparameters_metric_inner_loop = hyperparameters_metric_inner_loop+metric_fold                                                

                hyperparameters_metric_inner_loop = hyperparameters_metric_inner_loop/(fold_inner+1)                                          

                if hyperparameters_metric_inner_loop > best_hyperparameters_combinaion_dict['metric_inner_loop']:
                    best_hyperparameters_combinaion_dict['metric_inner_loop'] = hyperparameters_metric_inner_loop
                    best_hyperparameters_combinaion_dict['hyperparameters'] = hyperparameters

            best_hyperparameters = best_hyperparameters_combinaion_dict['hyperparameters']
            
            if 'vgg' in modalities:
                feature_values_imaging_dict = embedding_dict[(tuple(best_hyperparameters['pool_filter']), best_hyperparameters['pool_operator'])]
                train_features_imaging_values_outer = aggregate_embeddings(embedding_dict = feature_values_imaging_dict, keys = train_key_values_outer)
                train_features_outer = np.concatenate([train_features_outer, train_features_imaging_values_outer], axis = 1)
            
            predictor = XGBClassifier(
                        n_estimators = best_hyperparameters['n_estimators'],
                        max_depth = best_hyperparameters['max_depth'],
                        gamma = best_hyperparameters['gamma'],
                        min_child_weight = best_hyperparameters['min_child_weight'],
                        max_delta_step = best_hyperparameters['max_delta_step'],
                        subsample = best_hyperparameters['subsample'],
                        colsample_bytree = best_hyperparameters['colsample_bytree'],
                        scale_pos_weight = scale_pos_weight,
                        n_jobs= 8,
                        random_state = 373315
                        )    
            
            predictor.fit(X = train_features_outer, y = train_labels_outer)
            
            if 'vgg' in modalities:
                test_features_imaging_values_outer = aggregate_embeddings(embedding_dict = feature_values_imaging_dict, keys = test_key_values_outer) 
                test_features_outer = np.concatenate([test_features_outer, test_features_imaging_values_outer], axis = 1)
            
            results_outer_loop_dict = {
                'test_key_values_outer': str(test_key_values_outer),
                'y_true': bool(test_labels_outer),
                'y_pred': bool(predictor.predict(test_features_outer)),
                'y_pred_proba': predictor.predict_proba(test_features_outer),
                'idx': int(test_idx_outer),
                'fold': str(np.repeat(a=fold_outer, repeats=len(test_idx_outer))),
                'splitting_point': int(np.repeat(a=t, repeats=len(test_idx_outer))),
                'modality': str(np.repeat(a=modalities, repeats=len(test_idx_outer))),
                'best_hyperparameters': str(np.repeat(a=best_hyperparameters, repeats=len(test_idx_outer))),
              }   
             
            results_outer_loop_dict['y_pred_proba_positive'] = float(np.repeat(a=[p[1] for p in results_outer_loop_dict['y_pred_proba']], repeats=len(test_idx_outer)))
            final_results_preliminary_list.append(results_outer_loop_dict)
                
final_results = pd.DataFrame(final_results_preliminary_list)      